In [ ]:
import os
import pandas as pd
import json
import numpy as np
from sklearn.metrics import accuracy_score, mean_absolute_error, confusion_matrix

from pipeline import train_seq2seq_model, predict_scores_for_spans

project_root = os.path.abspath(os.path.join(os.getcwd(), "..", "..", ".."))
print("Project root:", project_root)

train_jsonl = os.path.join(project_root, "data", "processed", "training", "whatsapp_aspect_training_300.jsonl")
val_jsonl   = os.path.join(project_root, "data", "processed", "validation", "validation_gpt_50.jsonl")

# Span-level inputs from preprocessing
preproc_results_dir = os.path.join(project_root, "notebooks", "cs_scoring", "llm_preprocessing", "results")
flat_spans_path = os.path.join(preproc_results_dir, "b2b_feedback_attribute_spans_flat.csv")

# Where to store seq2seq outputs
results_dir = os.path.join(os.getcwd(), "results")
os.makedirs(results_dir, exist_ok=True)

print(train_jsonl, os.path.exists(train_jsonl))
print(val_jsonl, os.path.exists(val_jsonl))
print(flat_spans_path, os.path.exists(flat_spans_path))

In [ ]:
spans_df = pd.read_csv(flat_spans_path)
spans_df.head()

In [ ]:
models_to_train = [
    "google/flan-t5-base",
    "facebook/bart-base",
]
models_to_train

In [ ]:
trained_model_dirs = {}

for model_name in models_to_train:
    safe_name = model_name.replace("/", "_")
    output_dir = os.path.join(results_dir, safe_name)

    print(f"🚀 Training {model_name}")
    trainer, tokenizer, model = train_seq2seq_model(
        model_name=model_name,
        train_jsonl=train_jsonl,
        val_jsonl=val_jsonl,
        output_dir=output_dir,
        num_train_epochs=3,
        per_device_batch_size=8,
        learning_rate=5e-5,
    )

    trained_model_dirs[model_name] = output_dir

trained_model_dirs

In [ ]:
spans_predictions = {}

for model_name, model_dir in trained_model_dirs.items():
    print(f"\n🔮 Predicting with {model_name}")
    pred_df = predict_scores_for_spans(model_path=model_dir, spans_df=spans_df)

    spans_predictions[model_name] = pred_df

    out_csv = os.path.join(results_dir, f"{model_name.replace('/', '_')}_pred.csv")
    pred_df.to_csv(out_csv, index=False, encoding="utf-8-sig")

    print("Saved to:", out_csv)

spans_predictions

In [ ]:
val_records = []
with open(val_jsonl, "r", encoding="utf-8") as f:
    for line in f:
        val_records.append(json.loads(line))

val_df = pd.DataFrame(val_records)[["attribute", "text_span", "score"]]
val_df.head()

In [ ]:
print("===== FINAL MODEL METRICS =====\n")

for model_name, pred_df in spans_predictions.items():
    merged = val_df.merge(
        pred_df[["attribute", "text_span", "pred_score"]],
        on=["attribute", "text_span"],
        how="inner"
    )

    if len(merged) == 0:
        print(f"⚠️ No matching spans for model {model_name}")
        continue

    y_true = merged["score"].astype(int).values
    y_pred = merged["pred_score"].astype(int).values

    acc = accuracy_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred, labels=[1,2,3,4,5])

    print(f"\n🎯 MODEL: {model_name}")
    print(f"Total matches: {len(merged)}")
    print(f"Accuracy: {acc:.4f}")
    print(f"MAE: {mae:.4f}")
    print("Confusion Matrix (rows=true, cols=pred):")
    print(cm)
